In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import sys
from collections import Counter
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import multiprocessing
from multiprocessing import Pool
import itertools
from copy import deepcopy
import random

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, PReLU, GlobalMaxPooling2D
from keras.optimizers import Adam
from keras.utils import to_categorical, Sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.xception import Xception
from keras.applications.densenet import DenseNet121, DenseNet169
#import tensorflow as tf
from skimage.transform import AffineTransform, warp

from datetime import datetime
import pickle
import scipy.stats as stats
import os
import sklearn.metrics
import tensorflow_hub as hub

from efficientnet.keras import EfficientNetB6

Using TensorFlow backend.


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
y = pd.read_csv("data/train.csv")
yEval = pd.read_csv("data/test.csv")
classMap = pd.read_csv("data/class_map.csv")

In [4]:
labels = ["grapheme_root","vowel_diacritic","consonant_diacritic"]

In [5]:
y = y.set_index("image_id")

In [6]:
tables = [pq.read_table('data/train_image_data_{0}.parquet'.format(i)) for i in range(4)]
tables = [table.to_pandas() for table in tables]
df = pd.concat(tables)
df = df.set_index("image_id")
del tables

In [13]:
def affine_image(img):
    """

    Args:
        img: (h, w) or (1, h, w)

    Returns:
        img: (h, w)
    """
    # ch, h, w = img.shape
    # img = img / 255.
    if img.ndim == 3:
        img = img[0]

    # --- scale ---
    min_scale = 0.8
    max_scale = 1.2
    sx = np.random.uniform(min_scale, max_scale)
    sy = np.random.uniform(min_scale, max_scale)

    # --- rotation ---
    max_rot_angle = 7
    rot_angle = np.random.uniform(-max_rot_angle, max_rot_angle) * np.pi / 180.

    # --- shear ---
    max_shear_angle = 10
    shear_angle = np.random.uniform(-max_shear_angle, max_shear_angle) * np.pi / 180.

    # --- translation ---
    max_translation = 4
    tx = np.random.randint(-max_translation, max_translation)
    ty = np.random.randint(-max_translation, max_translation)

    tform = AffineTransform(scale=(sx, sy), rotation=rot_angle, shear=shear_angle,
                            translation=(tx, ty))
    transformed_image = warp(img, tform)
    assert transformed_image.ndim == 2
    return (transformed_image*255).astype(np.uint8)

In [7]:
size=(120, 120)

def calcRotate(img):
    detector = cv2.ORB_create()
    keypoints=detector.detect(img)
    descriptors=detector.compute(img,keypoints)
    angles=[]
    weights=[]
    for i in descriptors[0]:
        if i.angle!=-1:
            angles.append(i.angle)
            weights.append(i.response)
    if len(angles)==0:
        return 0
    else:
        return np.average(angles,weights=weights)

def upper(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[up:mid,:])==0:
            up=mid
        else:
            bottom=mid
    return max(up-mergin,0)

def lower(img,mergin=20):
    up=0
    bottom=img.shape[0]
    while bottom-up>2:
        mid=(up+bottom)//2
        if np.sum(img[mid:bottom,:])==0:
            bottom=mid
        else:
            up=mid
    return min(bottom+mergin,img.shape[0])

def lefter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,left:mid])==0:
            left=mid
        else:
            right=mid
    return max(left-mergin,0)

def righter(img,mergin=20):
    left=0
    right=img.shape[1]
    while right-left>2:
        mid=(left+right)//2
        if np.sum(img[:,mid:right])==0:
            right=mid
        else:
            left=mid
    return min(right+mergin,img.shape[1])

def transformImg(img,size=(255,255),training=True,clahe=False):
    ret2, img = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)
    img = 255-img
    if training:
        img=affine_image(img)
    if clahe:
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        img = clahe.apply(img)
    img = img[upper(img):lower(img),lefter(img):righter(img)]
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.resize(img, (size[0],size[1]))
    kernel = np.ones((3,3),np.float32)/9
    img = cv2.filter2D(img,-1,kernel)
    return img

def randomErase(img, prob=True):
    # random erasing
    # https://github.com/yu4u/cutout-random-erasing
    p = 0.5
    s_l = 0.02
    s_h = 0.4
    r_1 = 0.3
    r_2 = 1 / 0.3
    v_l = 0
    v_h = 255
    input_size=size[0]
    if prob==False or np.random.random()<p:

        while True:
            s = np.random.uniform(s_l, s_h) * input_size * input_size
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, input_size)
            top = np.random.randint(0, input_size)
            if left + w <= input_size and top + h <= input_size:
                break
        c = np.random.uniform(v_l, v_h, (h, w, 3))
        img[top : top + h, left : left + w, :] = c
    return img

In [8]:
class DataLoader(Sequence):
    def __init__(self,X,y,training,batch_size=64,size=(255,255),alpha=3,clahe=True):
        self.training = training
        self.batch_size=batch_size
        self.X=X
        self.y=y
        self.size=size
        self.alpha=alpha
        self.clahe=clahe

    def __len__(self):
        return int(np.ceil(self.X.shape[0] / self.batch_size))


    def __getitem__(self, idx):
        
        _imgs=self.X[idx * self.batch_size:(idx + 1) * self.batch_size,:,:]
        
            
        
        imgs=[]
        for img in _imgs:
            imgs.append(transformImg(img,size=self.size,training=self.training,clahe=self.clahe))
         
        
        
        imgs=np.asarray(imgs)
        
        
        ret_y=[]
        for label in labels:
            ret_y.append(to_categorical(self.y[idx * self.batch_size:(idx + 1) * self.batch_size][label],num_classes=len(set(y[label]))))
    
    
        #mix up
        if self.training :
            r = np.random.permutation(imgs.shape[0])
            imgs2=deepcopy(imgs)[r]
            grapheme=ret_y[0]
            vowel=ret_y[1]
            consonant=ret_y[2]
            grapheme2=deepcopy(grapheme)[r]
            vowel2=deepcopy(vowel)[r]
            consonant2=deepcopy(consonant)[r]
            ratio=np.random.beta(self.alpha,self.alpha,imgs.shape[0])
            ratio[ratio>1]=1
            ratio[ratio<0]=0
            imgs=np.tile(ratio,(3,*size,1)).T*imgs+np.tile((1-ratio),(3,*size,1)).T*imgs2
            grapheme=np.tile(ratio,(168,1)).T*grapheme+np.tile((1-ratio),(168,1)).T*grapheme2
            vowel=np.tile(ratio,(11,1)).T*vowel+np.tile((1-ratio),(11,1)).T*vowel2
            consonant=np.tile(ratio,(7,1)).T*consonant+np.tile((1-ratio),(7,1)).T*consonant2
            grapheme=grapheme.astype(np.float32)
            vowel=vowel.astype(np.float32)
            consonant=consonant.astype(np.float32)
            ret_y=[grapheme,vowel,consonant]
   
        if self.training:
            imgs = [randomErase(img) for img in imgs]
            pass
            
            
        imgs = np.asarray(imgs).astype(np.float32)/255.0
            

        return imgs, ret_y



In [9]:
def getMultiBased():
    model =  Xception(weights="imagenet", include_top=False,input_shape=(*size,3))
    x = model.output  
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    #x = Dense(1024, activation="relu")(x)
    grapheme = Dense(168, activation="softmax")(x)
    vowel = Dense(11,activation="softmax")(x)
    consonant = Dense(7, activation="softmax")(x)
    model = Model(inputs=model.input, outputs=[grapheme,vowel,consonant])
    return model

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df.values.reshape(-1,137,236),y,train_size=0.9,random_state=8000)
del df

In [11]:
early_stopping =  EarlyStopping(monitor='val_loss', min_delta=0.0, patience=20)
checkpoint = ModelCheckpoint(filepath="tmp-xce-epoch{epoch:04}.h5")

In [ ]:
batch_size=256
train_gen = DataLoader(X_train, y_train, training=True, batch_size=batch_size,size=size,clahe=True)
valid_gen = DataLoader(X_test, y_test, training=False, batch_size=batch_size,size=size,clahe=True)
#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
#with strategy.scope():
model = getMultiBased()
model.summary()
model.compile(optimizer=Adam(), metrics=["acc"], loss="categorical_crossentropy", loss_weights=[0.5,0.25,0.25])
print("compiled")
model.fit_generator(train_gen, validation_data=valid_gen, epochs=80, callbacks=[checkpoint], workers=64, use_multiprocessing=True)
model.save("multiXceptionMixup.h5")

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 120, 120, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 59, 59, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 59, 59, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 59, 59, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

Epoch 1/80
707/707 [==============================] - 672s 951ms/step - loss: 2.1790 - dense_4_loss: 3.2731 - dense_5_loss: 1.3614 - dense_6_loss: 0.8085 - dense_4_acc: 0.4130 - dense_5_acc: 0.6494 - dense_6_acc: 0.7766 - val_loss: 0.3586 - val_dense_4_loss: 0.5408 - val_dense_5_loss: 0.2348 - val_dense_6_loss: 0.1754 - val_dense_4_acc: 0.8741 - val_dense_5_acc: 0.9631 - val_dense_6_acc: 0.9597
Epoch 2/80
707/707 [==============================] - 621s 878ms/step - loss: 1.6898 - dense_4_loss: 2.4689 - dense_5_loss: 1.1360 - dense_6_loss: 0.6856 - dense_4_acc: 0.5811 - dense_5_acc: 0.7119 - dense_6_acc: 0.8147 - val_loss: 0.1849 - val_dense_4_loss: 0.3481 - val_dense_5_loss: 0.1210 - val_dense_6_loss: 0.1175 - val_dense_4_acc: 0.9068 - val_dense_5_acc: 0.9701 - val_dense_6_acc: 0.9672
Epoch 3/80
707/707 [==============================] - 622s 880ms/step - loss: 1.5195 - dense_4_loss: 2.1842 - dense_5_loss: 1.0590 - dense_6_loss: 0.6506 - dense_4_acc: 0.6028 - dense_5_acc: 0.7231 - dens

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



707/707 [==============================] - 623s 882ms/step - loss: 1.1182 - dense_4_loss: 1.5522 - dense_5_loss: 0.8220 - dense_6_loss: 0.5461 - dense_4_acc: 0.6648 - dense_5_acc: 0.7858 - dense_6_acc: 0.8540 - val_loss: 0.1451 - val_dense_4_loss: 0.2038 - val_dense_5_loss: 0.0650 - val_dense_6_loss: 0.0620 - val_dense_4_acc: 0.9466 - val_dense_5_acc: 0.9853 - val_dense_6_acc: 0.9827
Epoch 12/80
707/707 [==============================] - 626s 885ms/step - loss: 1.0941 - dense_4_loss: 1.5131 - dense_5_loss: 0.8108 - dense_6_loss: 0.5395 - dense_4_acc: 0.6720 - dense_5_acc: 0.7916 - dense_6_acc: 0.8580 - val_loss: 0.1152 - val_dense_4_loss: 0.1966 - val_dense_5_loss: 0.0645 - val_dense_6_loss: 0.0580 - val_dense_4_acc: 0.9466 - val_dense_5_acc: 0.9841 - val_dense_6_acc: 0.9835
Epoch 13/80
707/707 [==============================] - 625s 884ms/step - loss: 1.0739 - dense_4_loss: 1.4800 - dense_5_loss: 0.8007 - dense_6_loss: 0.5348 - dense_4_acc: 0.6774 - dense_5_acc: 0.7973 - dense_6_acc: 

Epoch 32/80
707/707 [==============================] - 624s 883ms/step - loss: 0.9049 - dense_4_loss: 1.2096 - dense_5_loss: 0.7195 - dense_6_loss: 0.4809 - dense_4_acc: 0.7414 - dense_5_acc: 0.8542 - dense_6_acc: 0.8976 - val_loss: 0.0976 - val_dense_4_loss: 0.2145 - val_dense_5_loss: 0.0616 - val_dense_6_loss: 0.0626 - val_dense_4_acc: 0.9520 - val_dense_5_acc: 0.9865 - val_dense_6_acc: 0.9861
Epoch 33/80
707/707 [==============================] - 625s 884ms/step - loss: 0.9030 - dense_4_loss: 1.2076 - dense_5_loss: 0.7190 - dense_6_loss: 0.4780 - dense_4_acc: 0.7416 - dense_5_acc: 0.8572 - dense_6_acc: 0.9000 - val_loss: 0.1061 - val_dense_4_loss: 0.1877 - val_dense_5_loss: 0.0548 - val_dense_6_loss: 0.0527 - val_dense_4_acc: 0.9537 - val_dense_5_acc: 0.9876 - val_dense_6_acc: 0.9864
Epoch 34/80
707/707 [==============================] - 625s 884ms/step - loss: 0.8966 - dense_4_loss: 1.1964 - dense_5_loss: 0.7158 - dense_6_loss: 0.4780 - dense_4_acc: 0.7435 - dense_5_acc: 0.8588 - d

In [15]:
model.evaluate(valid_gen)

79/79 [==============================] - 19s 237ms/step


[0.1342727541923523,
 0.245595321059227,
 0.05994708463549614,
 0.05561811849474907,
 0.9542421698570251,
 0.9891954064369202,
 0.9882991313934326]